In [2]:
import cv2
import glob
import numpy as np
from skimage.feature import hog
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Definir les carpetes de fruites fresques i podrides
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapple",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenorange"
}

# Inicialitzar llistes per emmagatzemar les característiques i etiquetes
data = []
labels = []

# Definir una funció per processar les imatges
def process_images(folder_dict):
    for folder, label in folder_dict.items():
        folder_path = f".gitignore/dataset/train/{folder}/*.png"
        for image_path in glob.glob(folder_path):
            img = cv2.imread(image_path)
            if img is None or img.shape[2] != 3:
                print(f"Imatge no vàlida o amb canals incorrectes: {image_path}")
                continue
            try:
                # Redimensionar la imatge
                resized_img = cv2.resize(img, (64, 32))
                # Convertir a tons de gris
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                # Extracció de característiques HOG
                fd = hog(
                    gray_img,
                    orientations=9,
                    pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2),
                    visualize=False,
                    feature_vector=True,
                    block_norm='L2-Hys'
                )
            except Exception as e:
                print(f"Error processant {image_path}: {e}")
                continue

            # Afegir les característiques i l'etiqueta a les llistes
            data.append(fd)
            labels.append(label)

# Processar les fruites fresques i podrides
process_images(freshfruits)
process_images(rottenfruits)

print("Total d'imatges processades:", len(data))
print("Total d'etiquetes:", len(labels))

# Codificació de les etiquetes
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(labels)

# Convertir les característiques a arrays de NumPy
X = np.array(data)
y = np.array(y_encoded)

# Dividir el conjunt de dades en entrenament i prova
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Total d'imatges processades: 10901
Total d'etiquetes: 10901


In [4]:
# Definir i entrenar el model One-vs-One amb kernel lineal
ovo_classifier = OneVsOneClassifier(SVC(kernel='linear', probability=True, random_state=42))
ovo_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred = ovo_classifier.predict(X_test)

# Avaluar el model
accuracy = accuracy_score(y_test, y_pred)
print("----- One-vs-One (Kernel Lineal) -----")
print(f"Accuracy: {accuracy:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred))

----- One-vs-One (Kernel Lineal) -----
Accuracy: 0.7203

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.72      0.70      0.71       339
 freshbanana       0.82      0.71      0.76       316
 freshorange       0.71      0.74      0.73       293
 rottenapple       0.66      0.70      0.68       469
rottenbanana       0.76      0.86      0.81       445
rottenorange       0.64      0.56      0.60       319

    accuracy                           0.72      2181
   macro avg       0.72      0.71      0.72      2181
weighted avg       0.72      0.72      0.72      2181

Matriu de Confusió:
[[238   0  26  60   2  13]
 [  2 224   3   1  86   0]
 [ 34   1 217  16   7  18]
 [ 36   0  31 330  10  62]
 [  5  47   1   3 384   5]
 [ 14   0  26  87  14 178]]


In [7]:
# One-vs-Rest (Kernel Lineal) - Model de Classificació

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model One-vs-Rest amb kernel lineal
ovr_classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=42))
ovr_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_ovr = ovr_classifier.predict(X_test)

# Avaluar el model
accuracy_ovr = accuracy_score(y_test, y_pred_ovr)
print("----- One-vs-Rest (Kernel Lineal) -----")
print(f"Accuracy: {accuracy_ovr:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_ovr, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_ovr))

----- One-vs-Rest (Kernel Lineal) -----
Accuracy: 0.7144

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.71      0.71      0.71       339
 freshbanana       0.81      0.70      0.75       316
 freshorange       0.69      0.72      0.71       293
 rottenapple       0.66      0.71      0.69       469
rottenbanana       0.76      0.87      0.81       445
rottenorange       0.64      0.51      0.57       319

    accuracy                           0.71      2181
   macro avg       0.71      0.70      0.71      2181
weighted avg       0.71      0.71      0.71      2181

Matriu de Confusió:
[[240   3  27  55   2  12]
 [  4 222   5   2  83   0]
 [ 33   3 211  16  11  19]
 [ 39   1  33 334   8  54]
 [  3  43   2   4 387   6]
 [ 17   1  27  93  17 164]]


In [9]:
# SVM amb Kernel RBF - Model de Classificació

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model SVM amb kernel RBF
rbf_classifier = SVC(kernel='rbf', gamma='scale', probability=True, random_state=42)
rbf_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_rbf = rbf_classifier.predict(X_test)

# Avaluar el model
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print("----- SVM amb Kernel RBF -----")
print(f"Accuracy: {accuracy_rbf:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_rbf, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_rbf))

----- SVM amb Kernel RBF -----
Accuracy: 0.7950

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.81      0.77      0.79       339
 freshbanana       0.90      0.82      0.86       316
 freshorange       0.78      0.82      0.80       293
 rottenapple       0.73      0.78      0.75       469
rottenbanana       0.84      0.93      0.88       445
rottenorange       0.72      0.61      0.66       319

    accuracy                           0.80      2181
   macro avg       0.80      0.79      0.79      2181
weighted avg       0.80      0.80      0.79      2181

Matriu de Confusió:
[[260   0  19  49   0  11]
 [  0 259   1   1  55   0]
 [ 22   2 240   9   7  13]
 [ 27   0  25 367   3  47]
 [  0  27   0   1 413   4]
 [ 11   0  23  77  13 195]]


In [8]:
# SVM amb Kernel Polinòmic - Model de Classificació

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Definir i entrenar el model SVM amb kernel polinòmic
poly_classifier = SVC(kernel='poly', degree=3, coef0=1, probability=True, random_state=42)
poly_classifier.fit(X_train, y_train)

# Predir sobre el conjunt de prova
y_pred_poly = poly_classifier.predict(X_test)

# Avaluar el model
accuracy_poly = accuracy_score(y_test, y_pred_poly)
print("----- SVM amb Kernel Polinòmic -----")
print(f"Accuracy: {accuracy_poly:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test, y_pred_poly, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test, y_pred_poly))

----- SVM amb Kernel Polinòmic -----
Accuracy: 0.8464

Informe de classificació:
              precision    recall  f1-score   support

  freshapple       0.84      0.84      0.84       339
 freshbanana       0.94      0.86      0.90       316
 freshorange       0.85      0.86      0.86       293
 rottenapple       0.81      0.82      0.82       469
rottenbanana       0.88      0.94      0.91       445
rottenorange       0.76      0.72      0.74       319

    accuracy                           0.85      2181
   macro avg       0.85      0.84      0.84      2181
weighted avg       0.85      0.85      0.85      2181

Matriu de Confusió:
[[286   0  14  30   0   9]
 [  0 273   0   0  42   1]
 [ 16   0 253   8   3  13]
 [ 28   0   9 384   2  46]
 [  1  18   0   3 420   3]
 [ 10   0  20  48  11 230]]


Conclusions Finals
Superioritat dels Kernels No Lineals: Els models amb kernels no lineals (RBF i Polinòmic) van superar significativament els models lineals (One-vs-One i One-vs-Rest), indicant que les relacions entre les característiques HOG i les classes de fruites no són linealment separables.

Millor Rendiment del Kernel Polinòmic: El Kernel Polinòmic va obtenir la millor precisió (84.64%), demostrant una millor capacitat per capturar les variacions intra-classe i les relacions complexes entre les característiques.

Reducció de Confusions: Els models no lineals van disminuir les confusions entre classes amb aspectes similars, millorant la precisió i la capacitat de generalització del model.

Recomanacions
Optimització d'Hiperparàmetres: Utilitzar tècniques com el Grid Search per trobar els millors paràmetres per a cada kernel.

Validació Creuada: Implementar validació creuada per assegurar la robustesa del model i evitar el sobreajust.

Aument de Dades: Considerar tècniques d'augment de dades per millorar la diversitat i quantitat de mostres, especialment en classes amb menor precisió.

Exploració d'Altres Característiques: Experimentar amb diferents mètodes d'extracció de característiques per capturar informació més discriminativa de les imatges.

En resum, la selecció adequada del kernel és crucial per al rendiment del model SVM en la classificació de fruites fresques i podrides. Els kernels no lineals, especialment el polinòmic, han demostrat una capacitat superior per manejar la complexitat dels dades, oferint millors mètriques de rendiment i una classificació més precisa.